IMPORTING DATASET

In [31]:
from scipy.io import arff
import pandas as pd

arff_file = arff.loadarff('speeddating.arff')
df = pd.DataFrame(arff_file[0])

df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

RENOMBRAR LES COLUMNES

DETERMINE REDUNDANT COLUMNS DATASET

In [32]:
# COLUMN d_age AND d_d_age CAN BE REMOVED BECAUSE IS THE RESULT OF TWO OTHER COLUMNS
df = df.drop(columns=['d_age', 'd_d_age'])

# samerace CAN BE REMOVED BEACUSE IS THE RESULT OF race AND race_o
df = df.drop(columns=['samerace'])

# WE MUST CHOOSE BETWEEN importance_same_race AND d_importance_same_race (same data but different presentation)
df = df.drop(columns=['d_importance_same_race'])

# WE MUST CHOOSE BETWEEN importance_same_religion AND d_importance_same_religion (same data but different presentation)
df = df.drop(columns=['d_importance_same_religion'])

# MUST BE REMOVED BECAUSE IS REPEATED 

df = df.drop(columns=[
    'd_pref_o_attractive', 'd_pref_o_sincere', 'd_pref_o_intelligence','d_pref_o_funny', 'd_pref_o_ambitious', 'd_pref_o_shared_interests',
    'd_attractive_o', 'd_sinsere_o', 'd_intelligence_o', 'd_funny_o', 'd_ambitous_o', 'd_shared_interests_o',
    'd_attractive_important', 'd_sincere_important', 'd_intellicence_important', 'd_funny_important', 'd_ambtition_important', 'd_shared_interests_important',
    'd_attractive', 'd_sincere', 'd_intelligence', 'd_funny', 'd_ambition',
    'd_attractive_partner', 'd_sincere_partner', 'd_intelligence_partner', 'd_funny_partner', 'd_ambition_partner', 'd_shared_interests_partner',
    'd_sports', 'd_tvsports', 'd_exercise','d_dining', 'd_museums', 'd_art', 'd_hiking', 'd_gaming', 'd_clubbing', 'd_reading', 'd_tv', 'd_theater', 'd_movies', 'd_concerts', 'd_music', 'd_shopping', 'd_yoga',
    'd_interests_correlate',
    'd_expected_happy_with_sd_people', 'd_expected_num_interested_in_me', 'd_expected_num_matches',
    'd_like', 'd_guess_prob_liked'])

# THE COLUMN MATCH MUST BE REMOVED BECAUSE IS THE RESULT OF OThER TWO
df = df.drop(columns=['match'])

df.to_csv('preprocessed.csv', index=False)

KNOW NUMBER OF NUMERICAL AND CATEGORIC

In [37]:
types = df.dtypes

num_categoricals = (types == 'object').sum()
num_numericals = ((types == 'int64') | (types == 'float64')).sum()

print ("Number of categorical variables: ", num_categoricals)
print ("Number of numerical variables: ", num_numericals)

Number of categorical variables:  7
Number of numerical variables:  58


DETECT MISSING VALUES

In [33]:
missing_values_percent = (df.isnull().sum() / len(df)) * 100

missing_data = pd.DataFrame({'Percent': missing_values_percent})
missing_data = missing_data.sort_values(by='Percent', ascending=False)

print(missing_data)

                                 Percent
expected_num_interested_in_me  78.515159
expected_num_matches           14.000955
shared_interests_o             12.843161
shared_interests_partner       12.735736
ambitous_o                      8.617809
ambition_partner                8.498448
met                             4.476009
funny_o                         4.296968
funny_partner                   4.177608
guess_prob_liked                3.688231
intelligence_o                  3.652423
intelligence_partner            3.533063
sinsere_o                       3.425639
sincere_partner                 3.306278
like                            2.864646
attractive_o                    2.530437
attractive_partner              2.411077
interests_correlate             1.885892
pref_o_shared_interests         1.539747
shared_interests_important      1.444259
pref_o_ambitious                1.277154
intelligence                    1.253282
ambition                        1.253282
funny           

DETECT OUTLIERS